In [7]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import datetime
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from math import *
from sklearn.model_selection import train_test_split

x_centre = 3760901.5068
y_centre = -19238905.6133

train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0'],axis=1)

tr = test.loc[test.x_exit.isnull()]

def conv(time_1):   
    sec1 = 3600*int(time_1[0:2]) + 60*int(time_1[3:5]) + int(time_1[6:8])                                                                                                                                
    return (sec1)

def distance(x1,x2,y1,y2):
    dist = np.sqrt(((x2-x1)**2)+((y2-y1)**2))
    return dist

def ecartTemps(t1,t2):
    ecart= t2-t1
    return ecart

def distanceCentre(x,y):
    dx = max(0,abs(x_centre-x)-10000)
    dy = max(0,abs(y_centre-y)-30000)
    return (dx**2+dy**2)

ecartTemps = np.vectorize(ecartTemps)

distance = np.vectorize(distance)
distanceCentre = np.vectorize(distanceCentre)
#vitesses négatives



test.vmin[test.vmin < 0] = np.NaN
test.vmean[test.vmean < 0] = np.NaN
test.vmax[test.vmax < 0] = np.NaN

train.vmin[train.vmin < 0] = np.NaN
train.vmean[train.vmean < 0] = np.NaN
train.vmax[train.vmax < 0] = np.NaN

#incohérences
train['vmean'][(train.time_entry==train.time_exit)] = 0
train['vmin'][(train.time_entry==train.time_exit)] = 0
train['vmax'][(train.time_entry==train.time_exit)] = 0

test['vmean'][(test.time_entry==test.time_exit)] = 0
test['vmin'][(test.time_entry==test.time_exit)] = 0
test['vmax'][(test.time_entry==test.time_exit)] = 0

train['vmax'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmin'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmean'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0

test['vmax'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmin'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmean'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0

#incohérences 2 

# train['vmean'][(train.time_entry!=train.time_exit)&(train.vmean == 0)] = np.NaN
# train['vmin'][(train.time_entry!=train.time_exit)&(train.vmin == 0)] = np.NaN
# train['vmax'][(train.time_entry!=train.time_exit)&(train.vmax == 0)] = np.NaN

# test['vmean'][(test.time_entry!=test.time_exit)&(test.vmean == 0)] = np.NaN
# test['vmin'][(test.time_entry!=test.time_exit)&(test.vmin == 0)] = np.NaN
# test['vmax'][(test.time_entry!=test.time_exit)&(test.vmax == 0)] = np.NaN

#train['vmax'][((train.x_entry!=train.x_exit)|(train.y_entry!=train.y_exit))&(train.vmax == 0)] = np.NaN
#train['vmin'][((train.x_entry!=train.x_exit)|(train.y_entry!=train.y_exit))&(train.vmin == 0)] = np.NaN
#train['vmean'][((train.x_entry!=train.x_exit)|(train.y_entry!=train.y_exit))&(train.vmean == 0)] = np.NaN

#test['vmax'][((test.x_entry!=test.x_exit)|(test.y_entry!=test.y_exit))&(test.vmax == 0)] = np.NaN
#test['vmin'][((test.x_entry!=test.x_exit)|(test.y_entry!=test.y_exit))&(test.vmin == 0)] = np.NaN
#test['vmean'][((test.x_entry!=test.x_exit)|(test.y_entry!=test.y_exit))&(test.vmean == 0)] = np.NaN

#outliers
train.vmean = train.vmean.loc[(train.vmean<90)]
train.vmax = train.vmean.loc[(train.vmax<90)]
train.vmin = train.vmean.loc[(train.vmin<90)]


#nan by mean
train['vmax']=train['vmax'].replace( np.NaN , train['vmax'].mean() )
train['vmin']=train['vmin'].replace( np.NaN , train['vmin'].mean() )
train['vmean']=train['vmean'].replace( np.NaN , train['vmean'].mean() )

test['vmax']=test['vmax'].replace( np.NaN , test['vmax'].mean() )
test['vmin']=test['vmin'].replace( np.NaN , test['vmin'].mean() )
test['vmean']=test['vmean'].replace( np.NaN , test['vmean'].mean() )

# trainTempVmean = train.loc[(train.vmean>0)]
# trainTempVmean = trainTempVmean.dropna()
# trainTempVmean = trainTempVmean.vmean.mean()

# trainTempVmax = train.loc[(train.vmax>0)]
# trainTempVmax = trainTempVmax.dropna()
# trainTempVmax = trainTempVmax.vmax.mean()

# trainTempVmin = train.loc[(train.vmin>0)]
# trainTempVmin = trainTempVmin.dropna()
# trainTempVmin = trainTempVmin.vmin.mean()

# testTempVmean = test.loc[(test.vmean>0)]
# testTempVmean = testTempVmean.dropna()
# testTempVmean = testTempVmean.vmean.mean()

# testTempVmax = test.loc[(test.vmax>0)]
# testTempVmax = testTempVmax.dropna()
# testTempVmax = testTempVmax.vmax.mean()

# testTempVmin = test.loc[(test.vmin>0)]
# testTempVmin = testTempVmin.dropna()
# testTempVmin = testTempVmin.vmin.mean()


# train['vmax']=train['vmax'].replace( np.NaN , trainTempVmax)
# train['vmin']=train['vmin'].replace( np.NaN , trainTempVmin)
# train['vmean']=train['vmean'].replace( np.NaN , trainTempVmean)
# test['vmax']=test['vmax'].replace( np.NaN , testTempVmax)
# test['vmin']=test['vmin'].replace( np.NaN , testTempVmin)
# test['vmean']=test['vmean'].replace( np.NaN , testTempVmean)

vec_conv =  np.vectorize(conv)
train.time_entry=vec_conv(train.time_entry)
train.time_exit=vec_conv(train.time_exit)

test.time_entry=vec_conv(test.time_entry)
test.time_exit=vec_conv(test.time_exit)

#moments
# train['nuit']= 0
# train['nuit'][(train.time_entry<28800)]=1
# train['matin']= 0
# train['matin'][(train.time_entry<43200)&(train.time_entry>28800)]=1
# train['aprem']= 0
# train['aprem'][(train.time_entry>=43200)]=1

# test['nuit']= 0
# test['nuit'][(test.time_entry<28800)]=1
# test['matin']= 0
# test['matin'][(test.time_entry<43200)&(test.time_entry>28800)]=1
# test['aprem']= 0
# test['aprem'][(test.time_entry>=43200)]=1

train['duree']=ecartTemps(train.time_entry,train.time_exit)
test['duree']=ecartTemps(test.time_entry,test.time_exit)

train_time_entry = train.time_entry
train_time_exit = train.time_exit
test_time_entry = test.time_entry
test_time_exit = test.time_exit

train = train.drop(['time_entry','time_exit'],axis=1)
test = test.drop(['time_entry','time_exit'],axis=1)

#train['distCenter']=distanceCentre(train.x_entry,train.y_entry)
#test['distCenter']=distanceCentre(test.x_entry,test.y_entry)

train['distCenter_entry']=distanceCentre(train.x_entry,train.y_entry)
test['distCenter_entry']=distanceCentre(test.x_entry,test.y_entry)

train['distCenter_exit']=distanceCentre(train.x_exit,train.y_exit)
test['distCenter_exit']=distanceCentre(test.x_exit,test.y_exit)

train['Ecart_dist']=train['distCenter_exit']-train['distCenter_entry']
test['Ecart_dist']=test['distCenter_exit']-test['distCenter_entry']

#angles

u = np.array([train.x_exit,train.y_exit])-np.array([train.x_entry,train.y_entry])
v = np.array([x_centre-train.x_entry,y_centre-train.y_entry])

angle_train = []
for i in range(u.shape[1]):
    scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
    u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
    v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
    angle_train.append(degrees(acos(scalar/(u_norm*v_norm))))
    
u = np.array([test.x_exit,test.y_exit])-np.array([test.x_entry,test.y_entry])
v = np.array([x_centre-test.x_entry,y_centre-test.y_entry])

angle_test = []
for i in range(u.shape[1]):
    scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
    u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
    v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
    angle_test.append(degrees(acos(scalar/(u_norm*v_norm))))

#application

train['angle']=angle_train
test['angle']=angle_test

# u = np.array([3770901.5068-x_centre,-19208905.6133-y_centre])

# v = np.array([x_centre-train.x_entry,y_centre-train.y_entry])
# angle_entry_train = []
# for i in range(v.shape[1]):
#     scalar = (np.dot([u[0],u[1]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm(u))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_entry_train.append(degrees(acos(scalar/(u_norm*v_norm))))

# v = np.array([x_centre-train.x_exit,y_centre-train.y_exit])
# angle_exit_train = []
# for i in range(v.shape[1]):
#     scalar = (np.dot([u[0],u[1]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm(u))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_exit_train.append(degrees(acos(scalar/(u_norm*v_norm))))

# v = np.array([x_centre-test.x_entry,y_centre-test.y_entry])
# angle_entry_test = []
# for i in range(v.shape[1]):
#     scalar = (np.dot([u[0],u[1]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm(u))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_entry_test.append(degrees(acos(scalar/(u_norm*v_norm))))
    
# v = np.array([x_centre-test.x_exit,y_centre-test.y_exit])
# angle_exit_test = []
# for i in range(v.shape[1]):
#     scalar = (np.dot([u[0],u[1]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm(u))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_exit_test.append(degrees(acos(scalar/(u_norm*v_norm))))

# #application

# train['angle_entry']=angle_entry_train
# test['angle_entry']=angle_entry_test
# train['angle_exit']=angle_exit_train
# test['angle_exit']=angle_exit_test

# #(3750901.5068+3770901.5068)/2 x centre
# # = 3760901.5068
# #(-19268905.6133+-19208905.6133)/2
# # = -19238905.6133
# Center = distance(train.x_entry,3760901.5068,train.y_entry,-19238905.6133)
# train['Center'] = Center

train['city_center'] = 0
train['city_center'][(train.x_exit>=3750901.5068) & (train.x_exit<=3770901.5068)&(train.y_exit>=(-19268905.6133)) & (train.y_exit<=(-19208905.6133))]=1

train['time_entry'] = train_time_entry 
train['time_exit'] = train_time_exit 
test['time_entry'] = test_time_entry 
test['time_exit'] = test_time_exit
# train.drop(['x_entry', 'x_exit','y_exit','y_entry'], axis=1, inplace=True)
# test.drop(['x_entry', 'x_exit','y_exit','y_entry'], axis=1, inplace=True)

In [8]:
train.drop(['distCenter_exit', 'Ecart_dist'], axis=1, inplace=True)
test.drop(['distCenter_exit', 'Ecart_dist'], axis=1, inplace=True)

In [18]:
# Y=train
# Y = Y.groupby('hash')['city_center'].apply(list)
# for i in range (Y.shape[0]):
#     Y[i]=Y[i][-1]

# Y=pd.DataFrame(Y)
# Y = pd.DataFrame(Y.values)

# Y.to_csv("Y.csv",index=False)

# train = train.drop(['city_center'],axis=1)
# train = train.drop(['trajectory_id'],axis=1)
# test = test.drop(['trajectory_id'],axis=1)


def transformOnLine(df):
    lesdataframes=[]

    for vincent in df.columns[1:]:
        print(vincent)
        k=df
        k = k.groupby('hash')[vincent].apply(list)

        if (vincent == 'distCenter_exit') | (vincent == 'x_exit') | (vincent == 'y_exit') | (vincent == 'Ecart_dist'):
            for i in range (k.shape[0]):
                k[i].reverse()
                k[i]=k[i][1:]      #x_exit y_exit
                
        elif (vincent == 'angle'):
            for i in range (k.shape[0]):
                try:
                    k[i]=k[i][-2]
                except(IndexError):
                    #k[i]=-1
                    k[i]=np.NaN
                    
#         elif (vincent == 'Ecart_dist'):
#             for i in range (k.shape[0]):
#                 k[i]=(np.cumsum(np.array(k[i][:-1]))).tolist()
                    
#                  k[i].reverse()
#                  k[i]=np.nanmean(np.array(k[i][1:]))

#         elif (vincent == 'vmin') | (vincent == 'vmax') | (vincent == 'vmean'):
#             for i in range (k.shape[0]):
#                 k[i].reverse()
#                 m=np.nanmean(np.array(k[i]))
# #                 M=max(k[i])
# #                 mi=min(k[i])
#                 k[i]=[m]
# #                 last=k[i][0]
# #                 if (last!=0):
# #                     if (len(k[i])>1):
# #                         k[i][0]=np.nanmean(np.array(k[i][1:]))
# #                     else:
# #                         k[i][0]=-1
                
        else :
            for i in range (k.shape[0]):
                k[i].reverse()


        k=pd.DataFrame(k)


        # expand df.time_entry into its own dataframe
        gouz = k[vincent].apply(pd.Series)

        # rename each variable is time_entry
        gouz = gouz.rename(columns = lambda x : vincent + str(x))
        
        if(vincent == 'distCenter_entry') | (vincent == 'angle'):# | (vincent == 'distCenter_exit') | (vincent == 'angle'):# | (vincent == 'Ecart_dist'):
            gouz = gouz.fillna(-1)
        else :#(vincent!='angle'):
            gouz = gouz.fillna(0)

#         if(vincent == 'Ecart_dist'):
#             gouz = gouz.fillna(0)
#         else:
#             gouz = gouz.fillna(-1)
        
        
        
        lesdataframes.append(gouz)

    r = pd.concat(lesdataframes,axis=1)
    r = r.reset_index()
    r = r.drop(['hash'],axis=1)
    #r.columns = np.arange(len(r.columns))
    
#     sup = r.vmax0.loc[r.vmax0>=0]
#     r.vmax0.loc[r.vmax0<0] = sup.mean()
#     sup = r.vmin0.loc[r.vmin0>=0]
#     r.vmin0.loc[r.vmin0<0] = sup.mean()
#     sup = r.vmean0.loc[r.vmean0>=0]
#     r.vmean0.loc[r.vmean0<0] = sup.mean()
    
    return r

In [19]:
X = transformOnLine(train)

vmax
vmin
vmean
x_entry
y_entry
x_exit
y_exit
duree
distCenter_entry
angle
time_entry
time_exit


In [20]:
X.to_csv("X_LSTM.csv",index=False)

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X=scaler.fit_transform(X)
#angle = pd.DataFrame(X)[216]
#X.drop(['angle0'],axis=1,inplace=True)
#X = pd.DataFrame(X).iloc[:,:216]

In [22]:
X = pd.DataFrame(X)

In [15]:
Vmax = X[X.columns[0:20]]
Vmin = X[X.columns[20:40]]
Vmean = X[X.columns[40:60]]
X_entry = X[X.columns[60:80]]
Y_entry = X[X.columns[80:100]]
X_exit = X[X.columns[100:119]]
Y_exit = X[X.columns[119:138]]
duree = X[X.columns[138:158]]
distCenter_entry = X[X.columns[158:178]]
time_entry = X[X.columns[178:198]]
time_exit = X[X.columns[198:218]]
# distCenter_exit = X[X.columns[178:197]]
# Ecart_dist = X[X.columns[197:216]]

In [16]:
columns = []
columns=[Vmax,Vmin,Vmean,X_entry,Y_entry,X_exit,Y_exit,duree,distCenter_entry,time_entry,time_exit]#,distCenter_exit,Ecart_dist]

NameError: name 'angle' is not defined

In [ ]:
from tqdm import tqdm

users = []
for i in tqdm(range(134063)):
    Data = []
    for each in columns:
        x = ((pd.DataFrame(each.iloc[i,:])).T)
        Data.append(x.values)
    for j in range(len(Data)):
        Data[j] = pd.DataFrame(Data[j])
    Data = pd.concat(Data)
    Data = Data.reset_index(drop=True)
    #Data.iloc[9,:] = (Data.iloc[9,:]).fillna(-1)
    Data = Data.fillna(-1) #pour les angles
    users.append(Data)
    #print((i/134063)*100)

In [ ]:
for i in tqdm(range(134063)):
    users[i] = users[i][users[i].columns[::-1]]

In [16]:
# import pickle

# with open('users.pkl', 'wb') as f:
#   pickle.dump(users, f)

In [ ]:
data = np.array([i.T.values for i in users])

In [ ]:
np.save('dataLSTM_v5', data)

In [37]:
#np.save('angleLSTM_v2', angle.values)

In [38]:
# Y = pd.read_csv('Y.csv')
# np.save('Y_LSTM', Y.values)